# Class 3: Treatment Effects Estimation under Unconfoundedness

- Unconfoundedness assumption.
- Stratified estimator.
- Propensity score.
- IPW estimator.
- Bootstrap.
- Balance checks.

## Unconfoundedness Assumption

In our hypothetical scenario, we were assuming that we could run an RCT where we could randomly assign our variable of interest (late status of a delivery in our example). In the real world, however, this is rarely possible. Moreover, the independence assumption:

$$\{Y_i(0), Y_i(1)\} \perp\!\!\!\!\perp W_i$$

is usually hard to defend. Suppose we have customers who live in "retirement" areas, more isolated higher-income zones in contrast to  urban, busy zones. These customers tend to be higher income and place larger and heavier orders in the platform due to a lack of other alternatives, and are more profitable for us. However, these larger orders tend to take a longer time to fulfill and deliver, and so we would expect to see these customers receiving disproportionately more late deliveries. If we also consider the fact that there might be less shoppers willing to fulfill the order, the likelihood of lateness is even higher. 

We will, however, assume that there exists some set of *observable, pre-treatment covariates* $X_i$ such that, after conditioning on $X_i$, the treatment $W_i$ is as good as random. This weaker assumption is referred to as **Unconfoundedness**.

$$\{Y_i(0), Y_i(1)\} \perp\!\!\!\!\perp W_i | X_i$$

The covariates $X_i$ are typically referred to as **confounders**. Without accounting for them, our simple treatment effect estimators will be biased. Unconfoundedness is still an untestable assumption, but since it is much weaker than full independence, it is also more defensible.

## Stratified Estimator

How do we account or "control" for $X_i$ in our treatment effect estimation? In its purest sense, unconfoundedness means that we have separate RCTs for each different value of $X_i$. That is, within each value of $X_i$, the units in the control group are acceptable counterfactuals of the units in the treatment group. Intuitively, it seems that we can first estimate the average treatment effect for each different value of $X_i$, that is, the **conditional average treatment effect** and then aggregate them to estimate the average treatment effect. This is easiest to understand when $X_i$ is discrete, that is, $X_i \in \mathcal{X}$ with $|\mathcal{X}| < \infty$ and we can clearly define its segments or **strata** (in our example above, living in a retirement area or not). The **stratified estimator** is defined as:

$$\hat{\tau}_{STRAT} = \sum_{x \in \mathcal{X}} \frac{n_x}{n} \hat{\tau}(x), \qquad {\hat{\tau}}(x) = \frac{1}{n_{x1}}\sum_{X_i = x, W_i = 1} Y_i - \frac{1}{n_{x0}}\sum_{X_i=x, W_i = 0} Y_i$$

This is essentially estimating, for each stratum $x$, the conditional average treatment effect using difference-in-means. Within each stratum, RCT assumptions hold, so difference-in-means is valid. Once we get valid estimates for each stratum, we aggregate them by weighting them according to the weight of the stratum in the sample, $\frac{n_x}{n}$. It can be shown that, if $\frac{n_x}{n}$ reasonably estimates the population weight, $\hat{\tau}_{STRAT}$ is unbiased for the average treatment effect $\tau$.

Remarkably, we are not restricted to use difference-in-means. $\hat{\tau}_{STRAT}$ remains a valid estimator for $\tau$ as long as the method used to obtain the estimate for $\tau(x)$ is valid. This includes using other covariates unrelated to the unconfoundedness for variance reduction in a linear regression setting!

We can obtain an expression for the asymptotic variance of this estimator invoking the Central Limit Theorem:

$$\sqrt{n}(\hat{\tau}_{STRAT} - \tau) \implies \mathcal{N}(0, \mathbb{V}_{STRAT}), \qquad \mathbb{V}_{STRAT} = \mathbb{V}[\tau(X_i)] +\mathbb{E}\left[\frac{\sigma^2_1(X_i)}{e(X_i)} + \frac{\sigma^2_0(X_i)}{1-e(X_i)}\right]$$

where $\sigma^2_1(X_i) = \mathbb{V}[Y_i(1)|X_i]$, $\sigma^2_0(X_i) = \mathbb{V}[Y_i(0)|X_i]$ and $e(X_i) = \mathbb{P}[W_i = 1|X_i]$. Note that the asymptotic variance does not depend on the number of strata!

## Propensity Score

The above derivations work well when $X_i$ is discrete and the strata are well defined such as our example above. But in many cases, the confounders are continuous variables, and it will be impossible to estimate something exactly as $\hat{\tau}_{STRAT}$ defined above, as we will not have access to enough observations for each possible value of $X_i$, less so if $X_i$ is multi-dimensional (think about age, income, historical activity on the platform), 

Luckily for us, in such situations, it suffices to condition on the **propensity score**, $e(X_i)$. The propensity score is the probability of receiving treatment conditioning on $X_i$:

$$e(x) = \mathbb{P}[W_i = 1|X_i = x]$$

It can be shown that:

$$\mathbb{P}[W_i = w|Y_i(w), e(X_i)] = \mathbb{P}[W_i = w|X_i]$$

The fact that it is enough to control for the propensity score makes obtaining a stratified estimator more appealing. We could partition our observations into strata delimited by the value of the propensity score and using our estimator $\hat{\tau}_{STRAT}$.

## Inverse-Propensity Weighting (IPW) Estimator

There is an alternative non-parametric estimator that uses propensity scores, simpler and less discretionary than the stratified estimator. This estimator is called **Inverse-Propensity Weighting (IPW)**, and we can think of it as a modification of the difference-in-means estimator where we weigh units differently based on the values of the propensity scores. The IPW estimator is defined as:

$$\hat{\tau}^*_{IPW} = \frac{1}{n}\sum_{i} \left( \frac{W_i Y_i}{e(X_i)} - \frac{(1-W_i)Y_i}{1-e(X_i)}\right)$$

We can also think of the IPW estimator as the average of "scores", $\Gamma_{i, IPW^*}$ (this is useful for a number of reasons we will see later):

$$\hat{\tau}^*_{IPW} = \frac{1}{n}\sum_{i} \Gamma_{i, IPW^*}, \qquad \Gamma_{i, IPW^*} =\left( \frac{W_i Y_i}{e(X_i)} - \frac{(1-W_i)Y_i}{1-e(X_i)}\right)$$

Let's see if this is a valid estimator for $\tau$:

\begin{align*}\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \mathbb{E}\left[\frac{1}{n}\sum_{i} \Gamma_{i, IPW^*}\right]\\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \mathbb{E}\left[\frac{W_i Y_i}{e(X_i)} - \frac{(1-W_i)Y_i}{1-e(X_i)}\right]\\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \mathbb{E}\left[\frac{W_i Y_i(1)}{e(X_i)} - \frac{(1-W_i)Y_i(0)}{1-e(X_i)}\right]\\
\end{align*}

Now let's use the unconfoundedness assumption:

\begin{align*}\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \mathbb{E}\left[\frac{W_i Y_i(1)}{e(X_i)} - \frac{(1-W_i)Y_i(0)}{1-e(X_i)}\right]\\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \mathbb{E}\left[\mathbb{E}\left[\frac{W_i Y_i(1)}{e(X_i)}|X_i\right] - \mathbb{E}\left[\frac{(1-W_i)Y_i(0)}{1-e(X_i)}|X_i\right]\right]\\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \mathbb{E}\left[\frac{\mathbb{E}\left[W_i Y_i(1)|X_i\right]}{e(X_i)} - \frac{\mathbb{E}\left[(1-W_i) Y_i(0)|X_i\right]}{1-e(X_i)}\right]\\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \mathbb{E}\left[\frac{\mathbb{E}\left[W_i|X_i\right] \mathbb{E}\left[Y_i(1)|X_i\right]}{e(X_i)} - \frac{\mathbb{E}\left[(1-W_i)|X_i\right] \mathbb{E}\left[Y_i(0)|X_i\right]}{1-e(X_i)}\right]\\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \mathbb{E}\left[\frac{\mathbb{P}\left[W_i|X_i\right] \mathbb{E}\left[Y_i(1)|X_i\right]}{e(X_i)} - \frac{\mathbb{P}\left[(1-W_i)|X_i\right] \mathbb{E}\left[Y_i(0)|X_i\right]}{1-e(X_i)}\right]\\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \mathbb{E}\left[\frac{e(X_i) \mathbb{E}\left[Y_i(1)|X_i\right]}{e(X_i)} - \frac{(1-e(X_i)) \mathbb{E}\left[Y_i(0)|X_i\right]}{1-e(X_i)}\right]\\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \mathbb{E}\left[\mathbb{E}\left[Y_i(1) - Y_i(0)|X_i\right]\right]\\
\mathbb{E}[\hat{\tau}^*_{IPW}] &= \tau\\
\end{align*}

Under unconfoundedness (and SUTVA), $\hat{\tau}^*_{IPW}$ is unbiased for $\tau$. Moreover, because $\hat{\tau}^*_{IPW}$ is the sum of $iid$ scores $\Gamma_{i, IPW^*}$, we can directly apply the Central Limit Theorem to obtain the limiting distribution:

$$\sqrt{n}(\hat{\tau}^*_{IPW} - \tau) \implies \mathcal{N}(0, \mathbb{V}_{IPW*}), \qquad \mathbb{V}_{IPW^*} = \mathbb{V}[\tau(X_i)] +\mathbb{E}\left[\frac{\sigma^2_1(X_i)}{e(X_i)} + \frac{\sigma^2_0(X_i)}{1-e(X_i)}\right] + \mathbb{E}\left[\frac{\left[\mu_0(X_i)+(1-e(X_i))\tau(X_i)\right]^2}{e(X_i)(1-e(X_i))}\right]$$

where $\mu_w(X_i)$ is the part of the potential outcome $Y_i(w)$ that can be explained with $X_i$, that is, $Y_i(w) = \mu_w(X_i) + \varepsilon_i(w)$ with $\varepsilon_i(w)$ being the ($iid$) random component.

Essentially, IPW is rebalancing our units by assigning more weight to those units whose treatment status is more "rare": among the units that received treatment, those with a lower propensity score (and hence with a lower chance of receiving treatment) will have a higher weight. This process attempts to undo the selection in the DGP.  

One important thing to note here is that IPW makes gives us a simple valid way to estimate treatment effects under unconfoundedness, but we pay a price in terms of variance. In a setting where $\hat{\tau}_{STRAT}$ is well-defined, it is straight-forward to see that:

$$V_{IPW^*} = V_{STRAT} + \mathbb{E}\left[\frac{\left[\mu_0(X_i)+(1-e(X_i))\tau(X_i)\right]^2}{e(X_i)(1-e(X_i))}\right]$$

so IPW actually performs worse than the stratified-estimator in such settings.

Another drawback of IPW is that it is very sensitive to extreme values of the propensity score. In fact, if for some value of $X_i$ the propensity score is $0$ or $1$, IPW can't be defined. We typically address this problem by requiring the propensity score to not have such extreme values, invoking what we refer to as an "overlap" assumption. That is, we assume that there exists an $\eta >0$ such that: 

$$\eta \leq e(x) \leq 1-\eta \text{ for all } x \in \mathcal{X}$$

This is called the "strong overlap" assumption and it will have implications in how we use IPW in practice. There are some estimators similar to IPW that require a weaker overlap assumption (referred to as "weak overlap"). Assuming strong overlap is fine for our purposes so whenever we think of "overlap", we will be referring to the strong overlap assumption. 

Let's see how this all works. Let's introduce a modification to the DGP from our previous exercise that lifts the RCT assumption.

\begin{align*}
i &= 1,\ldots, n\\
X_{ij} &\overset{iid}{\sim} \mathcal{U}(-1, 1), \; j \in \{1,2,3\}\\
e(X_i) &= 0.1 + 0.85 * \sqrt{\frac{\text{max}\{0,1+X_{2i}+X_{3i}\}}{3}}\\
W_i &\sim Ber(e(X_i))\\
Y_i &= \text{exp}(X_{1i} + X_{2i}) + \text{max}\{0,X_{3i}\} W_i  
\end{align*}

The average treatment effect in this setting is the same as before:

\begin{align*}
\tau &= \mathbb{E}[Y_i(1) - Y_i(0)]\\
\tau &= \mathbb{E}[\text{exp}(X_{1i} + X_{2i}) + \text{max}\{0,X_{3i}\} - \text{exp}(X_{1i} + X_{2i})]\\
\tau &= \mathbb{E}[\text{max}\{0,X_{3i}\}]\\
\tau &= \mathbb{E}[\text{max}\{0,X_{3i}\}|X_{3i} \geq 0]\mathbb{P}[X_{3i}\geq 0] + \mathbb{E}[\text{max}\{0,X_{3i}\}|X_{3i}<0]\mathbb{P}[X_{3i}<0]\\
\tau &= \underbrace{\mathbb{E}[\text{max}\{0,X_{3i}\}|X_{3i} \geq 0]}_{0.5}\underbrace{\mathbb{P}[X_{3i}\geq 0]}_{0.5} + 0\\
\tau &= 0.25
\end{align*}

In [ ]:
# YOUR CODE HERE

What would happen if we tried to estimate $\tau$ with difference-in-means?

In [ ]:
# YOUR CODE HERE

{'estimate': 0.8117806755938177,
 'std_error': 0.0072475491816618755,
 'ci_95': [0.7975754791977604, 0.825985871989875],
 'ci_size_perc': 3.4997621458939574,
 'p_value': 0.0}

What about linear regression, or Lin's estimator, where we control linearly for $X_i$?

In [ ]:
# YOUR CODE HERE

{'estimate': 0.2164353837541795,
 'std_error': 0.003791410518347556,
 'ci_95': [0.20900426573950343, 0.22386650176885559],
 'ci_size_perc': 6.8668236087645536,
 'p_value': 0.0}

Looks better, but bias is ~10% of the estimator. Let's try the two new estimators we proposed above, starting with $\hat{\tau}_{IPW}$. Note that because CLT holds, we can simply estimate the asymptotic variance with the sample variance of the scores $\Gamma_{i, IPW^*}$.  

In [ ]:
# YOUR CODE HERE

{'estimate': 0.25275380602641173,
 'std_error': 0.014355238171034942,
 'ci_95': [0.22461753921118324, 0.2808900728416402],
 'ci_size_perc': 22.263773003116203,
 'p_value': 2.1737884717986452e-69}

So much better! Let's compare this to $\hat{\tau}_{STRAT}$ using the propensity score as stratification variable and 20 strata.  

In [ ]:
# YOUR CODE HERE

## Bootstrap

Unlike the estimators we have computed so far, there is no immediate way to compute the variance of $\hat{\tau}_{STRAT}$ with a plugin estimator for $V_{STRAT}$. There is, however, a statistical technique that allows us to compute the distribution of almost any estimator through resampling methods: bootstrapping.

Bootstrapping is as conceptually simple as it is powerful. Given an estimator we can compute with a sample, bootstrapping allows to estimate the sampling distribution of the estimator by constructing $B$ new samples through resampling the original data with replacement $B$ times. In each of those new samples, we compute our estimator creating a distribution with the estimates. We then use that distribution to compute different statistics: mean, variance, confidence intervals, etc. Bootstrap. works almost everywhere assuming the observations are $iid$ and the sample is representative from the population we want to make inference of.

We want to use bootstrap to obtain an estimate of the variance and confidence intervals for $\hat{\tau}_{STRAT}$. There are a number of ways to do it, but we will obtain each using the most common method. For the variance, we will compute the sample variance of the estimator across our bootstrapped samples. For the (95%) confidence interval, we will grab the 2.5th and 97.5th percentiles of the bootstrapped distribution. Because we know the limiting distribution of $\hat{\tau}_{STRAT}$ (Normal), we will compare the percentile method of constructing the confidence intervals to simply using the bootstrapped variance.

Let's first evaluate whether bootstrap provides a reasonable estimate of the variance in the cases we can directly compute its expression.

In [ ]:
# YOUR CODE HERE

The bootstrapped variance and CI look pretty close to the ones obtained with the plugin estimators, suggesting that the bootstrap technique can provide a way to obtain the sampling distribution. Let's now use this learning and compute the variance and CI for $\hat{\tau}_{STRAT}$:

In [ ]:
# YOUR CODE HERE

It seems the empirical results are aligned with our derivations above: $V_{IPW^*} \geq V_{STRAT}$. Let's run simulations to compare their behavior under different values of $n$ and different number of strata.

In [ ]:
# YOUR CODE HERE

## Balance checks

Let's take a look at the rebalancing property of $e(X)$. A standard procedure when studying treatment effects is to run a **balance check**. A balance check analyzes the covariate distributions between the treatment and control groups. In a RCT, by design, the covariate distributions among the groups are the same. This amounts to say that the groups are comparable and they are good counterfactuals of each other. In practice, however, sampling errors and/or other shenanigans might threaten the balance of our setting. This is why it is always a good practice to check whether the covariates in our group are balanced or not. If they are not, then we might need to correct the imbalance.

There are multiple ways to help determine whether the sample is imbalanced. In this exercise we will focus on **standardized mean differences**. The concept is very simple: we compute the difference in means for each covariate in the treatment and control groups, and normalize by the standard error of the difference. For a given covariate $X_{ij}$:

$$S = \frac{\mu_{j1} - \mu_{j0}}{\sqrt{\sigma_{j1}^2 + \sigma_{j0}^2}}$$

And this can be estimated straight-forwardly with:

$$\hat{S} = \frac{\bar{X}_{j1} - \bar{X}_{j0}}{\sqrt{s_{j1}^2 + s_{j0}^2}}$$

where $s_{jw}^2$ is the sample variance of covariate $X_j$ for units in the treatment group $w$.

Let's simulate our RCT setting again and check the balance.

\begin{align*}
i &= 1,\ldots, n\\
X_{ij} &\overset{iid}{\sim} \mathcal{U}(-1, 1), \; j \in \{1,2,3\}\\
W_i &\overset{iid}{\sim} Ber(p)\\
Y_i &= \text{exp}(X_{1i} + X_{2i}) + 0.25 W_i  
\end{align*}

In [ ]:
# YOUR CODE HERE

As expected, covariates are balanced across groups in the RCT setting. What about in the observational dataset?

In [ ]:
# YOUR CODE HERE

Balance looks pretty bad. As a rule of thumb, covariates with a standardized difference below 0.10 can be considered balanced across groups.

But what would happen if we reweigh the observations using the (inverse of the) propensity score?

In [ ]:
# YOUR CODE HERE